# Simple ADK Agents

In [3]:
! pip install google-cloud-aiplatform[adk] --quiet --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.9/155.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# @title Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [6]:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}
STAGING_BUCKET = "gs://agent-engine-staging-bucket-anita" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "qwiklabs-gcp-02-8501559ee091" # @param {type:"string"}

print(PROJECT_ID)

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

qwiklabs-gcp-02-8501559ee091


In [1]:
# @title Test the agent locally
from google.adk.agents import Agent
from google.adk.agents import LlmAgent, BaseAgent
from google.adk.tools import google_search

# from google.adk.models.lite_llm import LiteLlm

AGENT_MODEL = "gemini-2.0-flash" # Can be a string for Gemini or a LiteLlm object

google_search_agent = LlmAgent(

    name="search_agent_v1",
    model=AGENT_MODEL,
    description=(
        "You can search Google and return answers from the Internet."
    ),
    instruction=(
        """You are a helpful Chatbot.
        If you don't know something, then Search Google for the answer."""
    ),
    tools=[google_search]
)



In [8]:
# @title Create an App that uses the agent

from vertexai.preview import reasoning_engines
app = reasoning_engines.AdkApp(
    agent=google_search_agent,
    enable_tracing=False,
)

from vertexai import agent_engines
remote_agent = agent_engines.create(
 app,
 requirements=["google-cloud-aiplatform[agent_engines,adk]"],
)

INFO:vertexai.agent_engines:Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.102.0', 'pydantic': '2.11.7'}
INFO:vertexai.agent_engines:The following requirements are appended: {'pydantic==2.11.7', 'cloudpickle==3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[agent_engines,adk]', 'pydantic==2.11.7', 'cloudpickle==3.1.1']
INFO:vertexai.agent_engines:Using bucket agent-engine-staging-bucket-anita
INFO:vertexai.agent_engines:Wrote to gs://agent-engine-staging-bucket-anita/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://agent-engine-staging-bucket-anita/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://agent-engine-staging-bucket-anita/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backing LRO: proje

In [10]:

# @title Send a message to the locally running agent
# @title Create a user session
session = app.create_session(user_id=user_id)

from IPython.display import Markdown, display

#test google search
for event in remote_agent.stream_query(
 user_id="agent-engine-test-user",
 message="Give me the news highlights in the world of science.",
):
    lastevent = event

display(Markdown(lastevent["content"]["parts"][0]["text"]))



Here are some of the recent science news highlights:

**Health and Medicine:**

*   **Heart Disease and Plastics:** A chemical widely used in plastics, DEHP, may have contributed to over 356,000 heart-related deaths globally in 2018, especially in Asia and the Middle East. This chemical is linked to inflammation in arteries and is found in items like food packaging.
*   **Diabetes Treatment:** MIT engineers have developed a tiny implantable device containing powdered glucagon that could revolutionize emergency treatment for people with Type 1 diabetes. The device can be triggered remotely, either manually or automatically.
*   **Alzheimer's Research:** A common sleeping pill could reduce Alzheimer's protein buildup, according to a study.
*   **Cancer Treatment:** A modified herpes virus has been found to shrink deadly skin cancers in a clinical trial.
*   **Creatine and the Brain:** Scientists are using focused ultrasound to deliver creatine past the blood-brain barrier, potentially reversing creatine deficiencies.
*   **Weight Loss and Health:** Experts are challenging the focus on BMI and dieting, suggesting that most people with higher body weight can't sustain long-term weight loss through lifestyle changes.
*   **Low Back Pain:** Walking frequently may reduce the likelihood of chronic low back pain.
*   **Nutrient Deficiencies:**There are 5 nutrients that might be lacking in your diet.

**Technology:**

*   **Quantum Computing:** Quantum materials with a 'hidden metallic state' could make electronics 1,000 times faster. Researchers controlled the conductive properties of a quantum material called 1T-TaS₂ by heating and cooling it.
*   **AI Hardware:** Scientists in Germany have engineered a vast spin waveguide network that processes information with far less energy, marking a significant step in AI hardware efficiency.
*   **3D Printing:** Scientists 3-D printed a tiny elephant inside a cell.

**Space and Earth Science:**

*   **Mars' Missing Water:** The mystery of Mars' missing water could be solved by the planet's varying tilt.
*   **Ancient Earth Impact:** An ancient Earth impact could help in the search for Martian life.
*   **Earth's Oldest Rocks:** Earth's oldest rocks may be at least 4.16 billion years old.
*   **Interstellar Comet:** Astronomers have traced the origin of a new interstellar comet.
*   **Earth's Spin:** Earth is spinning faster than usual, baffling scientists.
*   **Earth's Location:** Earth may be trapped inside a giant void in space, some scientists suggest.

**Biology and Environment:**

*   **Lemur Aging:** Lemurs age without inflammation, which could have implications for human health.
*   **New Viruses in Bats:** Two newly discovered viruses in bats are similar to Nipah and Hendra, raising concerns about potential outbreaks in humans.
*   **Wolf Pups:** Ice Age wolf pups once thought to be early dogs have been identified as wild wolves through DNA and chemical analysis.
*   **Bird Flu:** As bird flu evolves, keeping it out of farm flocks is getting harder.
*    **Trees and Climate:** Trees can't get up and walk away, but forests can.
*   **Tool Use by Killer Whales:** Killer whales have been filmed using tools for the first time.
*   **New Cell in Pythons:** A new cell discovered in pythons allows them to completely digest bones.
